In [1]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Replace 'path_to_your_zipped_file.zip' with the path to your zip file
# Replace 'path_to_destination_directory' with the path where you want to extract the files
!unzip /content/drive/MyDrive/DeepFashionDataset/segm.zip -d /content/drive/MyDrive/DeepFashionDataset/UnzippedFiles


In [3]:
import pandas as pd
import numpy as np

In [13]:
#Load datasets and rename columns
shapeDf = pd.read_csv('/content/drive/MyDrive/DeepFashionDataset/UnzippedFiles/labels/shape/shape_anno_all.txt', sep=" ", header=None)
shapeDf.columns = ['fileName', 'sleeveLength', 'lowerClothingLength', 'socks', 'hat', 'glasses', 'neckwear', 'wristWearing', 'ring', 'waistAccessories', 'neckline', 'isOuterClothingCardigan', 'isUpperClothingCoveringNaval']

FabricDf = pd.read_csv('/content/drive/MyDrive/DeepFashionDataset/UnzippedFiles/labels/texture/fabric_ann.txt', sep=" ", header=None)
FabricDf.columns = ['fileName', 'upperFabric', 'lowerFabric', 'outerFabric']

patternDf = pd.read_csv('/content/drive/MyDrive/DeepFashionDataset/UnzippedFiles/labels/texture/pattern_ann.txt', sep=" ", header=None)
patternDf.columns = ['fileName', 'upperPattern', 'lowerPattern', 'outerPattern']

                                    fileName  upperPattern  lowerPattern  \
0  MEN-Denim-id_00000080-01_7_additional.jpg             3             4   
1  MEN-Denim-id_00000089-01_7_additional.jpg             3             3   
2  MEN-Denim-id_00000089-02_7_additional.jpg             2             3   
3  MEN-Denim-id_00000089-03_7_additional.jpg             3             3   
4  MEN-Denim-id_00000089-04_7_additional.jpg             3             3   

   outerPattern  
0             7  
1             7  
2             7  
3             7  
4             7  
